## Task 2 (Detection & Counting of cars on main street)

In [1]:
# Necessary installs
!pip install pyttsx3
!pip install numpy
!pip install opencv-python
!pip install pypiwin32



 ### Counting cars moving towards city centre

In [2]:
import cv2

# Point of collision will be represented by 'Box' class
class Box:
    def __init__(self, start_point, width_height):
        self.start_point = start_point
        self.end_point = (start_point[0] + width_height[0], start_point[1] + width_height[1])
        self.counter = 0
        self.frame_countdown = 0

    def overlap(self, start_point, end_point):
        if self.start_point[0] >= end_point[0] or self.end_point[0] <= start_point[0] or \
                self.start_point[1] >= end_point[1] or self.end_point[1] <= start_point[1]:
            return False
        else:
            return True

# Create VideoCapture object & reading frames from input file
# Ensuring path to video is correct 
video=cv2.VideoCapture('Ex1_Files/Traffic_Laramie_2.mp4')

# Getting frames, fps and duration of video
frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
fps = int(video.get(cv2.CAP_PROP_FPS))
seconds = int(frames / fps)
minutes = seconds / 60

# Check if opening video is successful
if (video.isOpened()== False): 
  print("Error opening file!")

# Initialise initial frame to 'None'
initial_frame = None

# Building text string to count No. of cars
text = ""

# Initialise the value of cars to 0
cars = 0

# Creating box object using to count the cars
boxes = []
boxes.append(Box((500, 350), (10, 120)))

# Looping through function to continue reading frames until 'C' key is pressed
while True:
    check, frame = video.read()
    if check == True:
        # Smoothening (Noise Reduction)
        gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        blur_frame=cv2.GaussianBlur(gray_frame,(25,25),0)

        # 1st image = baseline image
        if initial_frame is None:
            initial_frame = blur_frame
            continue

        # Finding difference between baseline & frame images
        delta_frame=cv2.absdiff(initial_frame,blur_frame)

        # The difference between the initial frame and the current frame (the delta_frame) is transformed into a binary image.
        # When the pixel value is greater than 18,
        # The pixel will be rendered the colour white, otherwise, it is rendered the colour black 
        # threshold_frame=cv2.threshold(delta_frame,18,255, cv2.THRESH_BINARY)[1]
        threshold_frame=cv2.threshold(delta_frame,18,255, cv2.THRESH_BINARY)[1]

        # The kernel to applies to the morphology
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)) 
        closing = cv2.morphologyEx(delta_frame, cv2.MORPH_CLOSE, kernel)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        dilation = cv2.dilate(opening, kernel)
        retvalbin = cv2.threshold(dilation, 12, 255, cv2.THRESH_BINARY)[1]  

        # cv2.findContours() method allows us to identify the contours in the image
        (contours,_)=cv2.findContours(retvalbin,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        # Gets height of picture frame 
        height = 0
        if video.isOpened():
            height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

        # This loops over each contour 
        # Paints them on original image
        for c in contours:
            # contourArea() will filters out the small contours
            if cv2.contourArea(c) < 2500:
                continue
            M = cv2.moments(c)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            (x, y, w, h)=cv2.boundingRect(c)
            # If a car on main street is near the bottom half of the video frame, paint the contour over it
            if y > height / 2:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
                cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)


            # Constructing of texts strings
                cars = 0
                # Going through each box
                for box in boxes:
                    box.frame_countdown -= 1
                    if box.overlap((cX, cY), (x + w, y + h)):
                        if box.frame_countdown <= 0:
                            box.counter += 1
                        box.frame_countdown = 20
                    cars += box.counter

        text = "Cars: " + str(cars)
        # Setting up text strings being built
        cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

        for box in boxes:
            cv2.rectangle(frame, box.start_point, box.end_point, (255, 255, 255), 2)

        # We can use the different frames generated here.
        cv2.imshow('Video', frame)
        
        # cv2.imshow('Baseline image', initial_frame)
        # cv2.imshow('Gray Frame',gray_frame)
        # cv2.imshow('Delta frame', delta_frame)
        # cv2.imshow('dilation', dilation)
    
        if cv2.waitKey(1) & 0xFF == ord("c"):
            break
    
    else:
        # Outputs total number of cars
        print("Cars: " + str(cars))
        # Outputs cars per minute
        print("Cars per minute: " + str(round(cars/minutes, 2)))
        break

# Releasing video object after the loop is exited
video.release()

# Closing all windows
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

Cars: 4
Cars per minute: 2.29


-1